# Tailcut cleaning with PyWI-CTA

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jeremiedecock/pywi-cta-notebooks/master?filepath=tuto_2a_tailcut_cleaning.ipynb)

## Import required modules

This notebook requires PyWI-CTA for the I/O.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

import pywicta
import pywicta.data
from pywicta.io import geometry_converter
from pywicta.io.images import image_generator
from pywicta.io.images import plot_ctapipe_image
from pywicta.io.images import plot_hillas_parameters_on_axes

from pywicta.denoising.tailcut import Tailcut

print(pywicta.get_version())

## Get image

The next cells define the list of images to use in this notebook.

Images can be fetched from Fits files or from Simtel files.
Fits files are much lighter and much faster to process than Simtel files but they are specific to PyWI-CTA and thus you first have to generate them from Simtel files using [the following script](). Also, contrary to Simtel files, a Fits files contains only one "image" (i.e. an unique event viewed from one unique telescope).

In [ ]:
#pywicta.data.lst(["lst_gamma_r104_prod3b-north_t1_e1016403_660GeV", "lst_gamma_r104_prod3b-north_t1_e6905_87GeV"])
pywicta.data.lst()

In [ ]:
#CAM_ID = "ASTRICam"
#CAM_ID = "CHEC"
#CAM_ID = "DigiCam"
#CAM_ID = "FlashCam"
#CAM_ID = "NectarCam"
CAM_ID = "LSTCam"

IMG_PATH_LIST = pywicta.data.lst(["lst_gamma_r104_prod3b-north_t1_e1016403_660GeV"])

image_gen = image_generator(IMG_PATH_LIST,
                            cam_filter_list=[CAM_ID],
                            ctapipe_format=False)

img = next(image_gen)

In [ ]:
img.meta

## Plot

Images comming from FITS files are stored in 2D arrays (mainly because the Sparce2D wavelet transform expects images stored in 2D arrays).
But ctapipe uses a special 1D array representation for IACT images (defined in "geom" objects).
Thus conversion has to be done prior to use ctapipe functions on images comming from Fits files (using the "geometry_converter" function).

In [ ]:
geom1d = geometry_converter.get_geom1d(CAM_ID)

In [ ]:
img_input_1d = geometry_converter.image_2d_to_1d(img.input_image, CAM_ID)

disp = plot_ctapipe_image(img_input_1d,
                          geom1d,
                          title='Noised image',
                          norm='lin',
                          plot_axis=False)

## Setup the Tailcut cleaning

Fell free to change the following default setup to get a better understanding of the Tailcut cleaning...

In [ ]:
picture_thresh = 8
boundary_thresh = 4
pixels_clusters = "off"

## Clean the image with Tailcut

In [ ]:
tailcut = Tailcut()

tailcut_cleaned_img = tailcut.clean_image(img.input_image,
                                          high_threshold=picture_thresh,
                                          low_threshold=boundary_thresh,
                                          pixels_clusters_filtering=pixels_clusters,
                                          cam_id=CAM_ID)

In [ ]:
tailcut_cleaned_img_1d = geometry_converter.image_2d_to_1d(tailcut_cleaned_img, CAM_ID)

disp = plot_ctapipe_image(tailcut_cleaned_img_1d,
                          geom1d,
                          title='Clean image',
                          norm='lin',
                          plot_axis=False)

plot_hillas_parameters_on_axes(disp.axes,
                               tailcut_cleaned_img_1d,
                               geom1d,
                               hillas_implementation=2)